# Practice Spark

## 1. Install spark

In [0]:
# Install latest version of spark. If error, check the latest and replace "spark-2.4.4"
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"
import findspark
findspark.init()

## 2. Initiate SparkSession for Spark SQL

In [0]:
#The entry point to using Spark SQL is an object called SparkSession. 
#It initiates a Spark Application which all the code for that Session will run on.
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Learning_Spark") \
    .getOrCreate()

In [3]:
import numpy as np

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
data = spark.read.csv('/content/gdrive/My Drive/BigData/data_citibike_stations.csv',inferSchema=True, header=True)

In [7]:
#.count() returns the number of rows, .columns return a list of column names
data.count(), len(data.columns)

(843, 18)

In [8]:
data.columns

['station_id',
 'name',
 'short_name',
 'latitude',
 'longitude',
 'region_id',
 'rental_methods',
 'capacity',
 'eightd_has_key_dispenser',
 'num_bikes_available',
 'num_bikes_disabled',
 'num_docks_available',
 'num_docks_disabled',
 'is_installed',
 'is_renting',
 'is_returning',
 'eightd_has_available_keys',
 'last_reported']

In [9]:
data.show(6)

+----------+--------------------+----------+------------------+------------------+---------+--------------+--------+------------------------+-------------------+------------------+-------------------+------------------+------------+----------+------------+-------------------------+-------------------+
|station_id|                name|short_name|          latitude|         longitude|region_id|rental_methods|capacity|eightd_has_key_dispenser|num_bikes_available|num_bikes_disabled|num_docks_available|num_docks_disabled|is_installed|is_renting|is_returning|eightd_has_available_keys|      last_reported|
+----------+--------------------+----------+------------------+------------------+---------+--------------+--------+------------------------+-------------------+------------------+-------------------+------------------+------------+----------+------------+-------------------------+-------------------+
|       382|University Pl & E...|   5905.11|       40.73492695|      -73.99200509|       71

In [10]:
data.printSchema()

root
 |-- station_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- short_name: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- region_id: integer (nullable = true)
 |-- rental_methods: string (nullable = true)
 |-- capacity: integer (nullable = true)
 |-- eightd_has_key_dispenser: boolean (nullable = true)
 |-- num_bikes_available: integer (nullable = true)
 |-- num_bikes_disabled: integer (nullable = true)
 |-- num_docks_available: integer (nullable = true)
 |-- num_docks_disabled: integer (nullable = true)
 |-- is_installed: boolean (nullable = true)
 |-- is_renting: boolean (nullable = true)
 |-- is_returning: boolean (nullable = true)
 |-- eightd_has_available_keys: boolean (nullable = true)
 |-- last_reported: timestamp (nullable = true)



In [11]:
data.select("station_id","name","num_bikes_available").show(4)

+----------+--------------------+-------------------+
|station_id|                name|num_bikes_available|
+----------+--------------------+-------------------+
|       382|University Pl & E...|                  0|
|      3338|     2 Ave & E 99 St|                  0|
|      3512|       27 Ave & 4 St|                  0|
|      3709|     W 15 St & 6 Ave|                  0|
+----------+--------------------+-------------------+
only showing top 4 rows



In [12]:
data.select("station_id","name","num_bikes_available").show(4,truncate=False)

+----------+-----------------------+-------------------+
|station_id|name                   |num_bikes_available|
+----------+-----------------------+-------------------+
|382       |University Pl & E 14 St|0                  |
|3338      |2 Ave & E 99 St        |0                  |
|3512      |27 Ave & 4 St          |0                  |
|3709      |W 15 St & 6 Ave        |0                  |
+----------+-----------------------+-------------------+
only showing top 4 rows



In [13]:
data.describe(["num_bikes_available","num_bikes_disabled"]).show()

+-------+-------------------+------------------+
|summary|num_bikes_available|num_bikes_disabled|
+-------+-------------------+------------------+
|  count|                843|               843|
|   mean| 14.565836298932384|0.5693950177935944|
| stddev| 11.188256063195926|0.8613434732614029|
|    min|                  0|                 0|
|    max|                 66|                 6|
+-------+-------------------+------------------+



In [21]:
data.groupby("region_id").count().orderBy("count",ascending=False).show(5)

+---------+-----+
|region_id|count|
+---------+-----+
|       71|  792|
|       70|   51|
+---------+-----+



In [23]:
condition1 = data.region_id == 70
condition2 = (data.region_id.isNotNull())|(data.station_id.isNotNull())
data = data.filter(condition1).filter(condition2)
data.describe(["num_bikes_available","num_bikes_disabled"]).show()

+-------+-------------------+------------------+
|summary|num_bikes_available|num_bikes_disabled|
+-------+-------------------+------------------+
|  count|                 51|                51|
|   mean| 7.7254901960784315|0.5294117647058824|
| stddev|  4.647917518082906| 0.879839557566505|
|    min|                  0|                 0|
|    max|                 21|                 4|
+-------+-------------------+------------------+



## 3. PySpark for Word Counting